In [1]:
!pip install pgmpy


    100% |████████████████████████████████| 225kB 7.0MB/s 
    100% |████████████████████████████████| 1.3MB 12.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/27/d5/45/f9bd7e0f7d3752b706163ce2a0333d29855828c3e57a003640
Successfully built pgmpy
  Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2


In [0]:
import numpy as np
import pandas as pd
import csv
import re
import time
from time import time

from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from collections import defaultdict
from pgmpy.inference import VariableElimination
from pgmpy.estimators import HillClimbSearch
from pgmpy.sampling import BayesianModelSampling
from pgmpy.estimators import K2Score




In [0]:

def data_clean(df):
    new=[]
    for i, row in df.iterrows():
        newrow=[]
        for j, column in row.iteritems():
            column=str(column)        
            if(len(str(column))!=0):
                x=re.sub("%.*$", "", column)
                if(len(x)<5):
                    newrow.append(float(x)/100)
        new.append(newrow)
    df=pd.DataFrame(new)
    return df

#table read and cleaning
mar=pd.read_csv('Table2.csv')
mardf=data_clean(mar)

df3=pd.read_csv('Table3.csv')
df3=data_clean(df3)

df4=pd.read_csv('Table4.csv')
df4=data_clean(df4)

df5=pd.read_csv('Table5.csv')
df5=data_clean(df5)

df6=pd.read_csv('Table6.csv')
df6=data_clean(df6)

df7=pd.read_csv('Table7.csv')
df7=data_clean(df7)

df8=pd.read_csv('Table8.csv')
df8.iloc[6][5]=30.9
df8=data_clean(df8)





In [0]:
def marginal(c1,c2):
    y=np.outer(c1,c2)
    mask = (np.nan_to_num(y) != 0).any(axis=1)
    x=y[mask]
    x=np.transpose(x) 
    mask = (np.nan_to_num(x) != 0).any(axis=1)
    x=x[mask]
    return x


def conditional(cdf,a,b,x1,y1,g,h):
    new=[]   
    for i in range(0,len(cdf.columns)):
        x=cdf[i][g:h]
        mul=cdf[i][0]
        x=x.multiply(mul)
        new.append(x)
    y=np.transpose(new)
    y=np.array(y).reshape(a,b)
    return y


def createTable(new,df):
    finalDF=pd.DataFrame()
    
    for i in range(0,len(new)):
        x1,y1,g,h=new[i][0],new[i][1],new[i][2],new[i][3]
        x1-=1
        y1-=1
        x=marginal(mardf[x1],mardf[y1])
        a,b=x.shape
        y=(conditional(df,a,b,x1,y1,g,h))
        f,g=y.shape
        sub=np.subtract(x,y)
        a,b=sub.shape
        nu=a*b
        nums=np.transpose(sub)
        x=np.abs(nums).reshape(nu,1)
        x = pd.DataFrame.from_records(x)
        finalDF=pd.concat([finalDF,x],axis=1)
    return finalDF


    

In [8]:
finalDF = pd.DataFrame()
new2=[3,4,5,6,7,8]

l=[]

for i in new2:
    if(i==3):
        new6=[[1,2,1,6],[1,4,6,10],[1,6,10,15]]
        x3=createTable(new6,df3)
        finalDF=pd.concat([finalDF,x3],axis=1)
        #print(x3)
        
    if(i==4):
        new6=[[2,3,1,4],[2,5,4,8]]
        x4=createTable(new6,df4)
        finalDF=pd.concat([finalDF,x4],axis=1)
        #print(x4)
        
    if(i==5):
        new6=[[3,2,1,6],[3,5,6,10],[3,6,10,15]]
        x5=createTable(new6,df5)
        finalDF=pd.concat([finalDF,x5],axis=1)
        #print(x5)
        
    if(i==6):
        new6=[[4,1,1,5],[4,2,5,10],[4,6,10,15]]
        x6=createTable(new6,df6)
        finalDF=pd.concat([finalDF,x6],axis=1)
        #print(x6)
        
    if(i==7):
        new6=[[5,2,1,6],[5,3,6,9]]
        x7=createTable(new6,df7)
        finalDF=pd.concat([finalDF,x7],axis=1)
        #print(x7)
        
    if(i==8):
        new6=[[6,1,1,5],[6,2,5,10],[6,3,10,13],[6,4,13,17]]
        x8=createTable(new6,df8)
        finalDF=pd.concat([finalDF,x8],axis=1)

x=list(finalDF.sum(axis=0))
d = {"x2/x1":x[0], "x4/x1":x[1], "x6/x1":x[2], "x3/x2":x[3],'x5/x2':x[4],'x2/x3':x[5],'x5/x3':x[6],'x6/x3':x[7]
          ,'x1/x4':x[8],'x2/x4':x[9],'x6/x4':x[10],'x2/x5':x[11],'x3/x5':x[12],'x1/x6':x[13],'x2/x6':x[14],'x3/x6':x[15],'x4/x6':x[16]}
new={}
print(d)
x.sort(reverse=True)
result=[]

for i in d.keys():
    if(d[i]>0.12):
        result.append(d[i])
        new[i]=d[i]
    
#print(result)  
#print(new)  

{'x2/x4': 0.11569999999999997, 'x6/x1': 0.16015500000000005, 'x2/x6': 0.17531500000000003, 'x6/x3': 0.11324000000000001, 'x6/x4': 0.14346999999999996, 'x2/x1': 0.15977, 'x5/x3': 0.11551999999999997, 'x5/x2': 0.12926000000000004, 'x4/x6': 0.14307000000000003, 'x2/x3': 0.21875800000000006, 'x1/x4': 0.11957000000000002, 'x4/x1': 0.11943000000000004, 'x3/x2': 0.21852500000000002, 'x2/x5': 0.8561449999999998, 'x3/x6': 0.13903000000000004, 'x3/x5': 0.11670000000000004, 'x1/x6': 0.17684499999999995}


In [33]:
size=50000
model1 = BayesianModel()
#parent then child
G1 = ['x4 ,x6', 'x6 ,x3', 'x6 ,x2']

for i in G1:
    #print(e[0:2],e[4:7])
    model1.add_edge(i[0:2],i[4:7])


x4 = TabularCPD(variable='x4', variable_card=4,values=[[0.715],[0.105],[0.01],[0.17]])

x6 = TabularCPD(variable='x6', variable_card=5,values=df6.iloc[10:,], 
                    evidence=['x4'],
                   evidence_card=[4])
x2 = TabularCPD(variable='x2', variable_card=5,values=df8.iloc[5:10,], 
                    evidence=['x6'],
                   evidence_card=[5])
x3 = TabularCPD(variable='x3', variable_card=3,values=df8.iloc[10:13,], 
                    evidence=['x6'],
                   evidence_card=[5])


model1.add_cpds(x4,x6,x3,x2)

model1.edges()
inference = BayesianModelSampling(model1)
sample1 = inference.forward_sample(size, return_type='dataframe')
scorer1 = K2Score(sample1)
#print(sample)
print('Model 1 K2 Score: ' + str(scorer1.score(model1)))

Model 1 K2 Score: -222072.229299


In [34]:
#add 2 to all values eg x4 is df6

model2 = BayesianModel()
#parent then child
G2 = ['x6 ,x2','x2 ,x5', 'x6 ,x4','x4 ,x1']

for i in G2:
    model2.add_edge(i[0:2],i[4:7])



x6 = TabularCPD(variable='x6', variable_card=5,values=[[0.015],[0.32],[0.14],[0.315],[0.21]])

x2 = TabularCPD(variable='x2', variable_card=5,values=df8.iloc[5:10,], 
                    evidence=['x6'],
                   evidence_card=[5])
x4 = TabularCPD(variable='x4', variable_card=4,values=df8.iloc[13:17,], 
                    evidence=['x6'],
                   evidence_card=[5])

x1 = TabularCPD(variable='x1', variable_card=4,values=df6.iloc[1:5,], 
                    evidence=['x4'],
                   evidence_card=[4])

x5 = TabularCPD(variable='x5', variable_card=4,values=df4.iloc[4:8,], 
                    evidence=['x2'],
                   evidence_card=[5])

model2.add_cpds(x6,x4,x2,x1,x5)

model2.edges()

inference = BayesianModelSampling(model2)
sample2 = inference.forward_sample(size, return_type='dataframe')
scorer2  = K2Score(sample2)

print('Model 2 K2 Score: ' + str(scorer2.score(model2)))

Model 2 K2 Score: -271152.678885


In [35]:
model3 = BayesianModel()
#parent then child
G3 = ['x1 ,x6', 'x6 ,x3', 'x6 ,x2', 'x3 ,x5']

for i in G3:
    #print(e[0:2],e[4:7])
    model3.add_edge(i[0:2],i[4:7])


x1 = TabularCPD(variable='x1', variable_card=4
                    ,values=[[0.78],[0.015],[0.055],[0.15]])

x6 = TabularCPD(variable='x6', variable_card=5,values=df3.iloc[10:15,], 
                    evidence=['x1'],
                   evidence_card=[4])
x2 = TabularCPD(variable='x2', variable_card=5,values=df8.iloc[5:10,], 
                    evidence=['x6'],
                   evidence_card=[5])
x3 = TabularCPD(variable='x3', variable_card=3,values=df8.iloc[10:13,], 
                    evidence=['x6'],
                   evidence_card=[5])

x5 = TabularCPD(variable='x5', variable_card=4,values=df5.iloc[6:10,], 
                    evidence=['x3'],
                  evidence_card=[3])

model3.add_cpds(x1,x6,x3,x2,x5)

model3.edges()
inference = BayesianModelSampling(model3)
sample3 = inference.forward_sample(size, return_type='dataframe')
scorer3  = K2Score(sample3)
#print(sample)
print('Model 3 K2 Score: ' + str(scorer3.score(model3)))

Model 3 K2 Score: -275753.243498


In [36]:
model4 = BayesianModel()
#parent then child
G4 = ['x1 ,x6','x1 ,x2','x1 ,x4','x2 ,x3','x3 ,x5']

for i in G4:
    #print(e[0:2],e[4:7])
    model4.add_edge(i[0:2],i[4:7])

x1 = TabularCPD(variable='x1', variable_card=4
                    ,values=[[0.78],[0.015],[0.055],[0.15]])

x6 = TabularCPD(variable='x6', variable_card=5,values=df3.iloc[10:15,], 
                    evidence=['x1'],
                   evidence_card=[4])
x4 = TabularCPD(variable='x4', variable_card=4,values=df3.iloc[6:10,], 
                    evidence=['x1'],
                   evidence_card=[4])
x2 = TabularCPD(variable='x2', variable_card=5,values=df3.iloc[1:6,], 
                    evidence=['x1'],
                   evidence_card=[4])
x3 = TabularCPD(variable='x3', variable_card=3,values=df4.iloc[1:4,], 
                    evidence=['x2'],
                   evidence_card=[5])

x5 = TabularCPD(variable='x5', variable_card=4,values=df5.iloc[6:10,], 
                    evidence=['x3'],
                  evidence_card=[3])

model4.add_cpds(x1,x6,x4,x2,x3,x5)

model4.edges()
inference = BayesianModelSampling(model4)
sample4 = inference.forward_sample(size, return_type='dataframe')
scorer4  = K2Score(sample4)
#print(sample)
print('Model 4 K2 Score: ' + str(scorer4.score(model4)))

Model 4 K2 Score: -313107.503393


In [32]:
model5 = BayesianModel()
#parent then child
G5 = ['x6 ,x2','x6 ,x4','x4 ,x1','x2 ,x5','x5 ,x3']


for i in G5:
    #print(e[0:2],e[4:7])
    model5.add_edge(i[0:2],i[4:7])


x6 = TabularCPD(variable='x6', variable_card=5
                ,values=[[0.015],[0.32],[0.14],[0.315],[0.21]])

x2 = TabularCPD(variable='x2', variable_card=5,values=df8.iloc[5:10,], 
                    evidence=['x6'],
                   evidence_card=[5])

x4 = TabularCPD(variable='x4', variable_card=4,values=df8.iloc[13:,], 
                    evidence=['x6'],
                   evidence_card=[5])

x1 = TabularCPD(variable='x1', variable_card=4,values=df6.iloc[1:5,], 
                    evidence=['x4'],
                   evidence_card=[4])

x5 = TabularCPD(variable='x5', variable_card=4,values=df4.iloc[4:,  ], 
                    evidence=['x2'],
                   evidence_card=[5])

x3 = TabularCPD(variable='x3', variable_card=3,values=df7.iloc[6:,], 
                    evidence=['x5'],
                   evidence_card=[4])


model5.add_cpds(x2,x4,x6,x1,x5,x3)

model5.edges()
inference = BayesianModelSampling(model5)
sample5 = inference.forward_sample(size, return_type='dataframe')
scorer5  = K2Score(sample5)

print('Model 5 K2 Score: ' + str(scorer5.score(model5)))

Model 5 K2 Score: -313985.62665


In [0]:
#print(sample1)

In [85]:
x=sample1.groupby(["x4", "x6", "x3", "x2"]).size().reset_index(name="Count")
x1=float(x['Count'].max())
y=float(x['Count'].min())
#print(x1,y)
size=float(size)

maximum=x.iloc[x['Count'].argmax()]
minimum=x.iloc[x['Count'].argmin()]
#print(maximum,minimum)


print('The high probability of th is: {}'.format(round(x1/size,4)))
print('The low probablity of th is: {}'.format(round(y/size,4)))

The high probability of th is: 0.049
The low probablity of th is: 0.0001


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.
  


In [54]:

m = model1.to_markov_model()
m.nodes()
m.edges()

# Finding Model Inference

print("Bayesian Model Inference")
BayesianInferer = VariableElimination(model1)
s1 = time()
q1 = BayesianInferer.query(['x2'],evidence={'x6':4})['x2']
e1 = time()
time1 = e1 - s1
print("Model Query Time: "+str(round(time1,6)))
print(q1)

print("Markov Model Inference")
MarkovInferer = VariableElimination(m)
s2 = time()
q2 = MarkovInferer.query(['x2'],evidence={'x6':4})['x2']
e2 = time()
time2 = e2 - s2
print("Model Query Time: "+ str(round(time2,6)))
print(q2)

Bayesian Model Inference
Model Query Time: 0.00624
+------+-----------+
| x2   |   phi(x2) |
+======+===========+
| x2_0 |    0.2620 |
+------+-----------+
| x2_1 |    0.3090 |
+------+-----------+
| x2_2 |    0.0000 |
+------+-----------+
| x2_3 |    0.1670 |
+------+-----------+
| x2_4 |    0.2620 |
+------+-----------+
Markov Model Inference
Model Query Time: 0.005169
+------+-----------+
| x2   |   phi(x2) |
+======+===========+
| x2_0 |    0.2620 |
+------+-----------+
| x2_1 |    0.3090 |
+------+-----------+
| x2_2 |    0.0000 |
+------+-----------+
| x2_3 |    0.1670 |
+------+-----------+
| x2_4 |    0.2620 |
+------+-----------+


In [15]:
#Task 4
data=pd.read_csv('AND-Features(1).csv')
df1 = data.iloc[:,2:]
#print(df1)

hc = HillClimbSearch(df1, scoring_method=K2Score(data))
best_model = hc.estimate()
print(best_model.edges())

k2 = K2Score(df1)
print('Model 1 K2 Score: ' + str(k2.score(best_model)))


[('f3', 'f8'), ('f3', 'f9'), ('f3', 'f4'), ('f5', 'f9'), ('f5', 'f3'), ('f9', 'f1'), ('f9', 'f2'), ('f9', 'f4'), ('f9', 'f6'), ('f9', 'f7'), ('f9', 'f8')]
Model 1 K2 Score: -9462.70489237
